In [1]:
!pip install transformers peft torch datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 821.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login
# from huggingface_hub import notebook_login
# notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `test`


In [3]:
# 3 hours spent trying to figure out how to fine-tune with insufficient vram.
# the solution? just pay for a better colab GPU, apparently

In [4]:
import json
from datasets import Dataset

# Load JSON data
def read_in_data(json_file):
    with open(json_file, "r") as f:
        return json.load(f)

# Read and format dataset
JSON_FILE = "prompts_AHandfulOfDust_n1.json"
jdata = read_in_data(JSON_FILE)

dataset = Dataset.from_dict({"text": [d['prompt'] + d['ground_truth'] for d in jdata[5:35]]})

# Split dataset for training & evaluation
dataset = dataset.train_test_split(test_size=0.2)


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DialogueDiscriminator(nn.Module):
    def __init__(self, hidden_dim=256, input_dim=4096):  # Mistral hidden dim is 4096
        super(DialogueDiscriminator, self).__init__()
        self.encoder = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim,
                               batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(2 * hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        _, (h_n, _) = self.encoder(x)
        h = torch.cat((h_n[-2], h_n[-1]), dim=1)
        return self.fc(h)

# 2. Instantiate discriminator
input_dim = model.model.embed_tokens.embedding_dim  # Get from current transformer
hidden_dim = 256
discriminator = DialogueDiscriminator(hidden_dim=hidden_dim, input_dim=input_dim).to("cuda")
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-5)


In [7]:
def bce_loss(preds, targets):
  loss_fn = nn.BCELoss()
  return loss_fn(preds, targets)

In [8]:
print(model)
# 32 mistralattn layers, mlp

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [9]:
for param in model.parameters():
    param.requires_grad = False

for layer in model.model.layers[-2:]:
    for param in layer.parameters():
        param.requires_grad = True


trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 436224000


In [10]:
tokenizer.pad_token = tokenizer.unk_token

def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Labels should be the same as input_ids
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [11]:
def generate_response(prompt):
  model.eval()
  with torch.no_grad():
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)
generate_response("Hello!")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Hello! I'm a new member of the community and I'm excited to be here. I'm a writer and I love to read and write about science fiction and fantasy. I'm also a huge fan of video games and I love to play them. I'm always looking for new games to play and new stories to read. If you have any recommendations, please let me know!"

In [12]:
for i, (b, a) in enumerate(zip(before_weights, after_weights)):
    if not torch.allclose(b, a):
        print(f"Parameter {i} changed!")
    else:
      print(i)

NameError: name 'before_weights' is not defined

In [13]:
def embed_texts(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=256).to("cuda")
    with torch.no_grad():
        return model.model.embed_tokens(inputs['input_ids'])

In [14]:
from accelerate import Accelerator
from torch.utils.data import DataLoader
from torch.optim import AdamW
import os

lr = 1e-5
epochs = 3
batch_size = 4


accelerator = Accelerator(mixed_precision="no")
train_loader = DataLoader(tokenized_datasets['train'], batch_size=batch_size, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# thank you, adamw, for randomly deciding to replace entire matrices of weights
# with nan. thank you for not raising any sort of error when you did that, too.

model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
loss_history = []
model.train()
for epoch in range(epochs):
    for batch in train_loader:
        outputs = model(**batch)
        loss = outputs.loss

        if not torch.isfinite(loss):
          raise Exception("wee woo wee woo infinite loss alert")
        loss_history.append(loss.item())
        layer = model.model.layers[-1]

        accelerator.backward(loss)

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()
        # raise Exception("wawa")
print("loss", loss_history)

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 11795 has 14.73 GiB memory in use. Of the allocated memory 14.56 GiB is allocated by PyTorch, and 36.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:


for epoch in range(3):
    for batch in train_loader:
        real_texts = batch["ground_truth"]
        fake_texts = [generate_response(prompt) for prompt in batch['prompt']]

        # Embed
        real_embeds = embed_texts(real_texts)
        fake_embeds = embed_texts(fake_texts)

        # Discriminator training
        d_real = discriminator(real_embeds)
        d_fake = discriminator(fake_embeds)

        d_loss_real = bce_loss(d_real, torch.ones_like(d_real))
        d_loss_fake = bce_loss(d_fake, torch.zeros_like(d_fake))
        d_loss = d_loss_real + d_loss_fake

        discriminator_optimizer.zero_grad()
        d_loss.backward()
        discriminator_optimizer.step()

        # Generator training (language model)
        fake_embeds = embed_texts([generate_response(prompt) for prompt in batch['text']])
        g_fake = discriminator(fake_embeds)
        g_loss = bce_loss(g_fake, torch.ones_like(g_fake))

        model.zero_grad()
        g_loss.backward()
        optimizer.step()

        print(f"Epoch {epoch} | D_loss: {d_loss.item():.4f} | G_loss: {g_loss.item():.4f}")


In [ ]:
for j in jdata[:5]:
  prompt = j['prompt']
  print("===========================================")
  print(prompt)
  print(generate_response(prompt))

In [ ]:
# Save
model.save_pretrained("./gan_fine_tuned_model")
tokenizer.save_pretrained("./gan_fine_tuned_model")

# Load
# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(model_name)
# fine_tuned_model = PeftModel.from_pretrained(base_model, "./fine_tuned_model")


In [ ]:
!du -sh /content/gan_fine_tuned_model/model-00001-of-00003.safetensors

In [ ]:
!zip -r gan_fine_tuned_model.zip ./gan_fine_tuned_model